In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import sys
import json

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')

def ToDataFrame(h5file):
    entry_list = []
    group = h5file['result']
    for i, dataset_key in enumerate(group):
        entry = {}
        for j, item in enumerate(group[dataset_key].attrs):
            entry[item] = group[dataset_key].attrs[item]
        entry['grid'] = group[dataset_key][:]
        entry_list.append(entry)
    
    return pd.DataFrame(entry_list)

def ExtractPeakCoordinates(data):
    
    peak_list = []
    for i in data.index:
        image = data['grid'].iloc[i]

        peak_list.append(np.argwhere(image > np.mean(image) + 5 * np.std(image)))
        #peak_list.append(np.argwhere(image > 7.5))

    data['peaks'] = peak_list
    return data

def ConvertCoordinatesToDataUnits(data, angles, energies):
    
    peak_x = []
    peak_y = []
    
    for i in data.index:
        peak_coord = data['peaks'].iloc[i]
        #print(peak_coord)
        peak_x.append(energies[peak_coord[:, 1]])
        peak_y.append(angles[peak_coord[:, 0]])
        
    data['peak_x'] = peak_x
    data['peak_y'] = peak_y
    
    return data

def FitFunc(x, a, b): # x = 90 - \theta
    
    return a + b * x ** 2

def FitFuncInv(y, a, b):
    return + np.sqrt((y - a) / b)

def FitPolynomial(data, func):
    
    opt_param_list = []
    opt_cov_list = []
    for i in data.index:
        x_data = np.array(data.iloc[i]['peak_x'])
        y_data = np.array(data.iloc[i]['peak_y'])
        popt, copt = scipy.optimize.curve_fit(FitFunc, 90 - y_data, x_data - 18590)
        
        opt_param_list.append(popt)
        opt_cov_list.append(copt)
    
    data['fit_param'] = opt_param_list
    data['fit_cov'] = opt_cov_list
    
    return data
        
        


# get data

In [ ]:
result_list = os.listdir(RESULTPATH)
for result in result_list:
    if result.split('_')[0] == '210626':
        selected_result = result
h5file = h5py.File(os.path.join(RESULTPATH, selected_result), 'r')

data = ToDataFrame(h5file)


In [ ]:
h5file['result'].attrs['angles']

In [ ]:
h5file['result'].attrs['energies']

# extract coordinates of peak positions

In [ ]:
template_angles = np.sort(h5file['result'].attrs['angles'])
template_energies = np.sort(h5file['result'].attrs['energies'])
data = ExtractPeakCoordinates(data)
data = ConvertCoordinatesToDataUnits(data, template_angles, template_energies)

# plot example fit

In [ ]:
figname = '210714_polynomial_fit_to_energy_pitchangle_degeneracies_example1'

test_data = data[(data['signal-angle'] == 89.7) & (data['signal-energy'] == 18600)]
x_data = np.array(test_data['peak_x'].array[0])
y_data = np.array(test_data['peak_y'].array[0])
#print(x_data, y_data)
sig_E = test_data['signal-energy'].squeeze()
sig_a = test_data['signal-angle'].squeeze()
print(sig_E, sig_a)
popt, copt = scipy.optimize.curve_fit(FitFunc, 90 - y_data, x_data - 18590)

a = popt[0]
b = popt[1]
#c = popt[2]

E_fit = FitFunc(90 - y_data, a, b)

sns.set_theme(context='talk', style='white')
sns.set_style('ticks')
cmap = sns.color_palette('mako', as_cmap=True)

fig = plt.figure(figsize=(8,7))
ax = plt.subplot(1,1,1)

img = ax.imshow(test_data['grid'].squeeze(), 
           aspect='auto', 
           extent = (template_energies[0], template_energies[-1], template_angles[-1], template_angles[0]),
           cmap = cmap,
            interpolation='none'
            )
cbar = plt.colorbar(img, label='Matched Filter Score')
ax.set_xlabel('Template Energy (eV)')
ax.set_ylabel('Template Pitch Angle (deg)')
ax.set_title('Polynomial Fit to MF Score Degeneracies')

#ax.text(18594, 88.25, rf'Signal: $\theta$ = {sig_a}, E = {sig_E}', size=14, color='w')
plt.tight_layout()
#plt.savefig(os.path.join(PLOTPATH, figname + '.png'))

#plt.plot(E_fit + 18590, abs(-y_data), 'r')

#plt.savefig(os.path.join(PLOTPATH, figname + 'fit_curve.png'))

#plt.plot(x_data, y_data, 'r.')

#plt.savefig(os.path.join(PLOTPATH, figname + 'data_points.png'))

In [ ]:
data

# polynomial fit each curve

In [ ]:
data = FitPolynomial(data, FitFunc)

In [ ]:
data[data['signal-angle']==88.5]

# scatter plot of best fit parameters for each test signal

In [ ]:
signal_energies = np.sort(data['signal-energy'].unique())
signal_angles = np.sort(data['signal-angle'].unique())

In [ ]:
signal_angle_grid, signal_energy_grid = np.meshgrid(signal_angles, signal_energies)
p0_grid = np.zeros(signal_angle_grid.shape)
p1_grid = np.zeros(signal_angle_grid.shape)
p2_grid = np.zeros(signal_angle_grid.shape)
for i in range(signal_angle_grid.shape[0]):
    for j in range(signal_angle_grid.shape[1]):
            #print(data[(data['signal-angle'] == angle_grid.reshape(angle_grid.size)[i]) & (data['signal-energy'] == energy_grid.reshape(energy_grid.size)[i])])
            popt = data[(data['signal-angle'] == signal_angle_grid[i, j]) & (data['signal-energy'] == signal_energy_grid[i, j])]['fit_param']
            #print(angle_grid.reshape(angle_grid.size)[i], energy_grid.reshape(energy_grid.size)[i])
            #print(popt)
            p0_grid[i, j] = popt.squeeze()[0]
            p1_grid[i, j] = popt.squeeze()[1]
            #p2_grid[i, j] = popt.squeeze()[2]
            
            
fig = plt.figure(figsize=(13, 8))
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette(palette='mako', as_cmap=True)

#print(cmap)
ax = fig.add_subplot(1,1,1, projection='3d')
ax.view_init(30, -60)

#ax.plot3D(x,y,z, '.')
plot = ax.scatter(signal_angle_grid, signal_energy_grid, p0_grid, cmap=cmap )

plot.set_linewidth(.1)

ax.set_ylabel('Template Energy (eV)', labelpad = 12)
ax.set_xlabel('Template Pitch Angle (deg)', labelpad = 12)

figname = '210714_poly_fit_param0_raw'
plt.savefig(os.path.join(PLOTPATH, figname + '.png'))

fig = plt.figure(figsize=(13, 8))
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette(palette='mako', as_cmap=True)

#print(cmap)
ax = fig.add_subplot(1,1,1, projection='3d')
ax.view_init(30, -60)

#ax.plot3D(x,y,z, '.')
plot = ax.scatter(signal_angle_grid, signal_energy_grid, p1_grid, cmap=cmap )
ax.set_ylabel('Template Energy (eV)', labelpad = 12)
ax.set_xlabel('Template Pitch Angle (deg)', labelpad = 12)

figname = '210714_poly_fit_param1_raw'
plt.savefig(os.path.join(PLOTPATH, figname + '.png'))

# scatter plot of best fit parameters with outliers removed

In [ ]:
signal_energies = np.array(data['signal-energy'])
signal_angles = np.array(data['signal-angle'])
#print(signal_energies)
#print(signal_angles)
p0_vec = []
p1_vec = []
for i in range(len(signal_energies)):
    popt = data[
        (data['signal-angle'] == signal_angles[i]) & 
        (data['signal-energy'] == signal_energies[i])
    ]['fit_param']
    
    p0_vec.append(popt.squeeze()[0])
    p1_vec.append(popt.squeeze()[1])
    

p0_vec, p1_vec = np.array(p0_vec), np.array(p1_vec)

p1_cut_ind = np.argwhere(p1_vec > 55)

large_pitch_angle_large_energy_outlier_inds = np.argwhere(signal_angles > 89.7).squeeze()[np.isin(np.argwhere(signal_angles > 89.7).squeeze(), np.argwhere(p1_vec < 79).squeeze())]
p1_cut_ind = p1_cut_ind[~np.isin(p1_cut_ind, large_pitch_angle_large_energy_outlier_inds).squeeze()]

p0_vec_cut = p0_vec[p1_cut_ind].squeeze()
p1_vec_cut = p1_vec[p1_cut_ind].squeeze()

signal_energies_cut, signal_angles_cut = signal_energies[p1_cut_ind].squeeze(), signal_angles[p1_cut_ind].squeeze()

#print(len(signal_energies_cut))



fig = plt.figure(figsize=(13, 8))
sns.set_theme(context='talk', style='ticks')
#cmap = sns.color_palette(palette='mako', as_cmap=True)

ax = fig.add_subplot(1,1,1, projection='3d')
ax.view_init(0, -90)

#ax.plot3D(x,y,z, '.')
plot = ax.plot3D(signal_angles_cut, signal_energies_cut, p0_vec_cut, '.')

ax.set_ylabel('Template Energy (eV)', labelpad = 12)
ax.set_xlabel('Template Pitch Angle (deg)', labelpad = 12)

figname = '210714_poly_fit_param0_outliers_removed_diff_view'
plt.savefig(os.path.join(PLOTPATH, figname + '.png'))


fig = plt.figure(figsize=(13, 8))
sns.set_theme(context='talk', style='ticks')
#cmap = sns.color_palette(palette='mako', as_cmap=True)

ax = fig.add_subplot(1,1,1, projection='3d')
ax.view_init(0, -90)

#ax.plot3D(x,y,z, '.')
plot = ax.plot3D(signal_angles_cut, signal_energies_cut, p1_vec_cut, '.')

ax.set_ylabel('Template Energy (eV)', labelpad = 12)
ax.set_xlabel('Template Pitch Angle (deg)', labelpad = 12)

figname = '210714_poly_fit_param1_outliers_removed_diff_view'
plt.savefig(os.path.join(PLOTPATH, figname + '.png'))

# use curve fit to fit polynomial to data

In [ ]:
def FitP0(x, a, b, c, d):
    return a + b*x + c*x**2 + d*x**3

def FitP1(x, a, b, c, d):
    return a + b*x + c*x**2 + d*x**3 


#def FitP1(x, a, b):
#    return a + np.log(b * x)

signal_energies_unique = np.sort(data['signal-energy'].unique())
#signal_angles = np.sort(data['signal-angle'].unique())

p0_parameters = []
p1_parameters = []
for energy in signal_energies_unique:
    
    curve_inds = np.argwhere(signal_energies_cut == energy).squeeze()
    
    curve_energy = energy
    curve_angles = signal_angles_cut[curve_inds]
    
    p0_curve_vals = p0_vec_cut[curve_inds]
    p1_curve_vals = p1_vec_cut[curve_inds]
    
    
    popt0, copt0 = scipy.optimize.curve_fit(FitP0, curve_angles, p0_curve_vals)
    popt1, copt1 = scipy.optimize.curve_fit(FitP1, curve_angles, p1_curve_vals)
    #print(popt0, popt1)
    p0_parameters.append(popt0)
    p1_parameters.append(popt1)

#print(p0_parameters)

'''
popt0, copt0 = scipy.optimize.curve_fit(FitP0, signal_angles_cut, p0_vec_cut)
popt1, copt1 = scipy.optimize.curve_fit(FitP1, signal_angles_cut, p1_vec_cut)

curve_angles = np.linspace(88, 90, 201)
curve_z = FitP0(curve_angles, *popt0)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

#ax.plot3D(curve_angles, curve_energies, curve_z, '.')

ax.plot(signal_angles_cut, p0_vec_cut, 'k.')
ax.plot(curve_angles, curve_z)
plt.show()

curve_z = FitP1(curve_angles, *popt1)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

#ax.plot3D(curve_angles, curve_energies, curve_z, '.')

ax.plot(signal_angles_cut, p1_vec_cut, 'k.')
ax.plot(curve_angles, curve_z)
plt.show()
'''

# compare predicted degeneracy curve to data

In [ ]:
figname = '210714_fit_to_fit_fails_to_extrapolate_example'

energy_index = 4

test_energy = np.sort(data['signal-energy'].unique())[energy_index]
test_pa = 88.5

test_data = data[(data['signal-angle'] == test_pa) & (data['signal-energy'] == test_energy)]
x_data = np.array(test_data['peak_x'].array[0])
y_data = np.array(test_data['peak_y'].array[0])
#y_data = np.linspace(test_pa, test_pa + 0.1, 11)
#print(x_data, y_data)
sig_E = test_data['signal-energy'].squeeze()
sig_a = test_data['signal-angle'].squeeze()
print(sig_E, sig_a)

a = FitP0(test_pa, *p0_parameters[i])
b = FitP1(test_pa, *p1_parameters[i])
#c = popt[2]

fit_curve_energies = np.linspace(0, 10, 21)
fit_curve_pa = 90 - FitFuncInv(fit_curve_energies, a, b)

#print(E_fit)
sns.set_theme(context='talk', style='white')
sns.set_style('ticks')
cmap = sns.color_palette('mako', as_cmap=True)

fig = plt.figure(figsize=(8,7))
ax = plt.subplot(1,1,1)

img = ax.imshow(test_data['grid'].squeeze(), 
           aspect='auto', 
           extent = (template_energies[0], template_energies[-1], template_angles[-1], template_angles[0]),
           cmap = cmap,
            interpolation='none'
            )
cbar = plt.colorbar(img, label='Matched Filter Score')
ax.set_xlabel('Template Energy (eV)')
ax.set_ylabel('Template Pitch Angle (deg)')
ax.set_title('Polynomial Fit to MF Score Degeneracies')

ax.text(18594, 88.25, rf'Signal: $\theta$ = {sig_a}, E = {sig_E}', size=14, color='w')
plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, figname + '.png'))

plt.plot(18590 + fit_curve_energies, fit_curve_pa, 'r')

#plt.savefig(os.path.join(PLOTPATH, figname + 'fit_curve.png'))

plt.plot(x_data, y_data, 'r.')

#plt.savefig(os.path.join(PLOTPATH, figname + 'data_points.png'))

# plot fit curves

In [ ]:
curve_angles = np.linspace(88, 90, 201)


sns.set_theme(context='talk', style='whitegrid')
#cmap = sns.color_palette(palette='mako', as_cmap=True)

#ax = fig.add_subplot(1,1,1, projection='3d')
#ax.view_init(30, -60)

for i, energy in enumerate(signal_energies_unique):
    
    curve_inds = np.argwhere(signal_energies_cut == energy).squeeze()
    
    #curve_energy = energy
    data_angles = signal_angles_cut[curve_inds]
    p0_data_vals = p0_vec_cut[curve_inds]
    
    #curve_energies = energy * np.ones(curve_angles.size)
    
    curve_z = FitP0(curve_angles, *p0_parameters[i])
    
    fig = plt.figure(figsize=(8,5))
    ax = fig.add_subplot(1,1,1)
    ax.plot(curve_angles, curve_z)
    #ax.plot3D(curve_angles, curve_energies, curve_z, '.')
    
    ax.plot(data_angles, p0_data_vals, 'k.')
    plt.show()
#plot = ax.plot3D(signal_angles_cut, signal_energies_cut, p0_vec_cut, 'k.')

for i, energy in enumerate(signal_energies_unique):
    
    curve_inds = np.argwhere(signal_energies_cut == energy).squeeze()
    
    #curve_energy = energy
    data_angles = signal_angles_cut[curve_inds]
    p1_data_vals = p1_vec_cut[curve_inds]
    
    #curve_energies = energy * np.ones(curve_angles.size)
    
    curve_z = FitP1(curve_angles, *p1_parameters[i])
    
    fig = plt.figure(figsize=(8,5))
    ax = fig.add_subplot(1,1,1)
    ax.plot(curve_angles, curve_z)
    #ax.plot3D(curve_angles, curve_energies, curve_z, '.')
    
    ax.plot(data_angles, p1_data_vals, 'k.')
    #ax.set_xscale('log')
    plt.show()

In [ ]:
figname = '210628_polynomial_fit_params_mf_degeneracies'

def smooth(x, kernel):
    return np.convolve(x, kernel, mode='same')

p0_list = []
p1_list = []
angle_grid = []
for i in range(data.shape[0]):
        popt = data.iloc[i]['fit_param']
        
        #print(popt)
        p0_list.append(popt.squeeze()[0])
        p1_list.append(popt.squeeze()[1])
        angle_grid.append(data.iloc[i]['signal-angle'])
        
        #p2_list.append(popt[2])
angle_grid = np.array(angle_grid)
p0_list = np.array(p0_list)
p1_list = np.array(p1_list)
sns.set_theme(context='talk', style='whitegrid', palette='deep')
#sns.axes_style(style='whitegrid')
#sns.set_style('ticks')

kernel1 = np.ones(2) / 2
kernel2 = np.ones(2) / 2
fig, axs = plt.subplots(1, 2, figsize = (16,6), sharex=True)

axs[0].plot(angle_grid, p0_list, '.')


#axs[0].plot(angles, smooth(smooth(p0_list, kernel1), kernel2))
#axs[0].plot(angles, scipy.interpolate.interp1d(angles, p0_list, kind='cubic')(angles))

axs[0].set_ylim(-250, 50)
axs[0].set_ylabel(r'$p_0$')
axs[0].set_xlabel('Signal Pitch Angle (deg)')

axs[0].text(89.15, -170, r'$y=p_0+p_2x^2$')

axs[1].plot(angle_grid, p1_list, '.')
axs[1].set_ylim(-25, 100)
axs[1].set_ylabel(r'$p_2$')


axs[1].text(89.15, 12, r'$y=p_0+p_2x^2$')
axs[1].set_xlabel('Signal Pitch Angle (deg)')
#axs[2].plot(angles, p2_list)
#axs[2].set_ylim(-100, 100)

fig.suptitle('Best-Fit Polynomial Parameters')

#plt.savefig(os.path.join(PLOTPATH, figname+'.png'))


axs[0].plot(angle_grid[np.where(p1_list < 40)[0]], p0_list[np.where(p1_list < 40)[0]], '.')

axs[1].plot(angle_grid[np.where(p1_list < 40)[0]], p1_list[np.where(p1_list < 40)[0]], '.')

#plt.savefig(os.path.join(PLOTPATH, figname+'_outliers_highlighted.png'))

In [ ]:
np.where(p1_list < 40)[0]

# plot interpolation of best-fit polynomial parameters outliers removed

In [ ]:
figname = '210628_polynomial_fit_params_mf_degeneracies_3rd_order_fit_to_params'

p0_cut = p0_list[np.where(p1_list > 40)[0]]
p1_cut = p1_list[np.where(p1_list > 40)[0]]
angles_cut = angles[np.where(p1_list > 40)[0]]

fig, axs = plt.subplots(1, 2, figsize = (16,6), sharex=True)

def FitP0(x, a, b, c, d):
    return a + b*x + c*x**2 + d*x**3

def FitP1(x, a, b, c, d):
    return a + b*x + c*x**2 + d*x**3

popt0, copt0 = scipy.optimize.curve_fit(FitP0, angles_cut, p0_cut)
popt1, copt1 = scipy.optimize.curve_fit(FitP1, angles_cut, p1_cut)

axs[0].plot(angles_cut, p0_cut, '.')
axs[0].plot(angles_cut, FitP0(angles_cut, *popt0))
axs[0].set_ylabel(r'$p_0$')
axs[0].set_xlabel('Signal Pitch Angle (deg)')

axs[1].plot(angles_cut, p1_cut, '.')
axs[1].plot(angles_cut, FitP1(angles_cut, *popt1))
axs[1].set_ylabel(r'$p_2$')
axs[1].set_xlabel('Signal Pitch Angle (deg)')


fig.suptitle('Fit 3rd Order Polynomial to Best-Fit Polynomial Parameters')
plt.savefig(os.path.join(PLOTPATH, figname+'.png'))

In [ ]:
plt.hist(peak_energies)